Imports

In [2]:
import gensim.downloader as api
import pandas as pd
import numpy as np
from IPython.core.display import HTML
import time
import sys

In [2]:
# animate loading dataset...
done = False
#here is the animation
def animate():
    while not done:
        sys.stdout.write('\rloading the model ... |')
        time.sleep(0.1)
        sys.stdout.write('\rloading the model ... /')
        time.sleep(0.1)
        sys.stdout.write('\rloading the model ... -')
        time.sleep(0.1)
        sys.stdout.write('\rloading the model ... \\')
        time.sleep(0.1)
    sys.stdout.write('\rDone!     ')

Load "word2vec-google-news-300" pre-trained embedding model

In [1]:
#info = api.info()  # show info about available models/datasets
#print(info)

# obtain pre-trained Word2Vec model
model_name = "word2vec-google-news-300"
animate()
model = api.load(model_name)
done = True

print("Test purpose - finding similar word")
print(model.most_similar("enormously", topn=3))

NameError: name 'animate' is not defined

Load "synonyms.csv" dataset

In [79]:
# Load synonyms.csv dataset
df = pd.read_csv("synonyms.csv")

# display data frame
HTML(df.to_html())

Compute the closest synonym for each word in "synonms.csv" dataset using Similarity method

In [78]:
# new dataframe created to store closest synonyms with the question word 
# which is to be found from system model
new_df = pd.DataFrame(columns=['question', 'answer', 'guess', 'label'])

# iterate each data in dataframe by rows
for index, row in df.iterrows():
    
    # create dictionary to store accuracy value 
    # for similarity between question-word and each available guess-words
    accuracy_dict = {}
    
    # assign each elements in row to appropriate variables
    question = row["question"]
    answer = row["answer"]
    guess_0 = row["0"]
    guess_1 = row["1"]
    guess_2 = row["2"]
    guess_3 = row["3"]
    
    # if question-word and answer-word from synonyms.csv found in word2vec model:
    if (question in model) and (answer in model):
        if guess_0 in model:
            accuracy = model.similarity(question, guess_0)  # calculate cosine similarity
            accuracy_dict[guess_0] = accuracy               # append both word and accuracy to dictionary
        if guess_1 in model:
            accuracy = model.similarity(question, guess_1)
            accuracy_dict[guess_1] = accuracy
        if guess_2 in model:
            accuracy = model.similarity(question, guess_2)
            accuracy_dict[guess_2] = accuracy
        if guess_3 in model:
            accuracy = model.similarity(question, guess_3)
            accuracy_dict[guess_3] = accuracy
        
        # sort the accuracy dictionary by keys with higher accuracy value (descending)
        sorted_accuracy_tuples = sorted(accuracy_dict.items(), key=lambda item: item[1], reverse=True)
        sorted_accuracy_dict = {key: value for key, value in sorted_accuracy_tuples}
        #print(sorted_accuracy_dict)
        
        # obtain the guess word with highest accuracy value (first element in sorted dictionary!)
        guess_word = next(iter(sorted_accuracy_dict))
        
        # check if word guessed by system is equal to the answer in synonyms.csv:
        if guess_word == answer:
            label = "correct"
        else:
            label = "wrong"
        
        # append all the data (question-word, answer-word, guess-word by system, label) to new dataframe
        new_df.loc[index] = [question, answer, guess_word, label]
        
    
    # else if question-word and answer-word not found in word2vec model:
    else:
        label = "guess"
        guess_word = ""
        new_df.loc[index] = [question, answer, guess_word, label]

# display new dataframe
HTML(new_df.to_html())

# write new dataframe obtained into new CSV file
file_name = model_name + "-details.csv"
new_df.to_csv(file_name)